# **1. tsv -> KoBERT 데이터셋 형식으로 변환**

## **1-1. 라이브러리 불러오기**

In [ ]:
## for from kobert_transformers.utils import get_tokenizer
!pip install kobert-transformers

In [ ]:
## for from kobert.pytorch_kobert import get_pytorch_kobert_model
!pip install mxnet
!pip install transformers
!pip install torch

In [ ]:
## for from kobert.pytorch_kobert import get_pytorch_kobert_model
#!pip install onnxruntime==1.8.0
!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT
!pip install boto3

In [ ]:
## for import gluonnlp as nlp
!pip install gluonnlp

In [ ]:
# import modules

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp

import numpy as np

from tqdm import tqdm, tqdm_notebook
from kobert_transformers.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# for 데이터프레임 내 non-string을 string으로 변환
import pandas as pd
from transformers import BertModel, BertTokenizer
from kobert_tokenizer import KoBERTTokenizer

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# set device: PyTorch에서 GPU를 사용하기 위한 설정을 지정
device = torch.device("cuda:0")
print(torch.cuda.is_available())


True


## **1-2. tsv 형식 데이터셋 불러오기**

In [ ]:
#tsv 파일
dataset_train = nlp.data.TSVDataset('/content/gdrive/MyDrive/crimefin/train.tsv', field_indices=[1, 0])
dataset_test = nlp.data.TSVDataset('/content/gdrive/MyDrive/crimefin/test.tsv', field_indices=[1, 0])

In [ ]:
#데이터셋 확인
dataset_train[30]

['[ 부천 소사 경찰서 전경 경기 남부 경찰청 제공 아들 에게 폭행 을 당해 치아 가 부러진 채 맨발 로 도망친 대 노인 이 구조 됐 다 부천 소사 경찰서 는 존 속상해 혐의 로 대 남성 A 씨 를 입건 했 다고 일 밝혔 다 A 씨 는 지난 일 오전 시분 께 소사본동 주거지 에서 아버지 B 씨 를 폭행 했 다 B 씨 는 맨 발 로 도망쳐 나와 에 신고 했 고 경찰 은 주거지 인근 에서 치아 가 부러진 채 피 를 흘리 고 쓰러져 있 는 B 씨 를 발견 해 구급대 에 인계 했 다 이후 경찰 은 주거지 로 출동 해 A 씨 로부터 범행 사실 을 자백 받 고 그 를 현행범 체포 했 다 경찰 조사 에서 A 씨 는 “ 아버지 가 아무런 이유 없이 어머니 에게 화 를 내 아버지 를 때렸 다 ” 는 취지 로 진술 한 것 으로 알려졌 다 한편 경찰 은 B 씨도 A 씨 를 상대 로 폭력 을 행사 한 혐의 로 입건 해 조사 를 진행할 예정 이 다 ]',
 '0']

## **1-3. KoBERT 데이터셋 형식으로 변환**

### **1-3-1. BERTDataset 선언 | 파라미터 설정**

In [70]:
class BERTDataset(Dataset):
  # 클래스로 객체를 생성할 때 입력되는 인자들을 처리하는 초기화 함수
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # i번째 아이템(데이터와 레이블)을 반환
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

     # 전체 데이터셋의 크기(데이터 개수)를 반환: PyTorch DataLoader의 전체 배치 개수 등 계산에 사용됨
    def __len__(self):
        return (len(self.labels))

In [87]:
# SET Parameters: 이걸 왜 벌써 설정..?

MAX_LEN = 64
BATCH_SIZE = 16
WARMUP_RATIO = 0.1
NUM_EPOCHS = 10
MAX_GRAD_NORM = 1
LOG_INTERVAL = 200
LEARNING_RATE = 5e-5

### **1-3-2. BERTDataset 형식 변환**

In [88]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-mv0oux6o/kobert-tokenizer_e460cfed3067496ba0cc96f1d64e4471
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-mv0oux6o/kobert-tokenizer_e460cfed3067496ba0cc96f1d64e4471
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-1d270630
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-1d270630
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.15.18-py2.py3-none-any.whl (129 kB)
  Using cached mxnet-1.7.0.post2-py2.py3-none-manylinux2014_x86_64.whl (54.7 MB)
INFO: pip is looking at multiple versi

In [89]:
# tokenize 2번째
# BERT 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok=tokenizer.tokenize

using cached model. /content/KoBERT/.cache/kobert_v1.zip
using cached model. /content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [90]:
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, MAX_LEN, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, MAX_LEN, True, False)

In [91]:
data_train[1]

(array([   2, 1073,  517, 7095, 1674,  517, 6330,  517, 6079, 4064,  517,
        6116,  889, 3220, 4360, 7044,  517, 6116, 5130, 5010, 4965, 1864,
        3205, 2779,  517, 6116, 3579, 4924,  517, 5760, 2123, 2018,  517,
        5760, 4069, 6079, 4689, 6132, 4924, 2029, 3135, 1771,  695,  342,
         311,  517, 6116, 1526, 6113, 2029, 5194,  517, 6896, 3133, 6573,
        3205, 3647, 2779, 1772, 2872, 3854, 1562, 1848,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [92]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=BATCH_SIZE, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=BATCH_SIZE, num_workers=5)

# **2. KoBERT 모델링**

In [93]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [94]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * WARMUP_RATIO)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [95]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

# **3. 모델 학습**

In [96]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % LOG_INTERVAL == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-96-f5ee0939fbb5>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/5 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8307069540023804 train acc 0.1875
epoch 1 train acc 0.2875


<ipython-input-96-f5ee0939fbb5>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

...

epoch 9 test acc 0.5555555555555556


  0%|          | 0/5 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 1.497087836265564 train acc 0.0
epoch 10 train acc 0.6625


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 10 test acc 0.5555555555555556


# **4. 예측**

In [97]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, MAX_LEN, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=BATCH_SIZE, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("보이스피싱이 아닙니다.")
            elif np.argmax(logits) == 1:
                test_eval.append("보이스피싱입니다.")


        print(">> 회원님의 상황은 " + test_eval[0])

In [99]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("사용자님의 상황을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

사용자님의 상황을 입력해주세요 : 엄마가 핸드폰을 잃어버려 수리비가 필요하니 현금을 국민은행 계좌로 부치라고 했습니다
>> 회원님의 상황은 보이스피싱이 아닙니다.




KeyboardInterrupt: ignored